In [1]:
%%local
!echo "anaconda" | kinit edill

Password for edill@DEMO.ANACONDA.COM: 


In [52]:
%%configure -f
{"driverMemory": "8G", 
 "driverCores": 4,
 "executorMemory": "8G",
 "executorCores": 4,
 "numExecutors": 10
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
95,application_1561571912988_0191,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
95,application_1561571912988_0191,pyspark,idle,Link,Link,✔


In [53]:
sc

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-95>

In [54]:
import datetime as dt
import pyspark.sql.functions as F

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
start = '2016-10-28'
end = '2019-09-02'
start_dt = dt.datetime.strptime(start, '%Y-%m-%d')
end_dt = dt.datetime.strptime(end, '%Y-%m-%d')
start_dt, end_dt

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(datetime.datetime(2016, 10, 28, 0, 0), datetime.datetime(2019, 9, 2, 0, 0))

In [56]:
(end_dt - start_dt).days

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1039

In [57]:
def daterange(start_date, end_date):
    for n in range( int( (end_date - start_date).days)):
        yield start_date + dt.timedelta(n)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [58]:
dates = list(daterange(start_dt, end_dt))
len(dates)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1039

In [59]:
datestrings = [d.strftime('%Y-%m-%d') for d in dates]

datestrings[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'2016-10-28'

In [60]:
bucket = 's3://edill-data/cleaned-logs'
def date_to_path(ymd):
    y, m, d = ymd.split('-')
    return '{bucket}/{y}/{m}/{ymd}.parquet'.format(bucket=bucket, y=y, m=m, ymd=ymd)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [61]:
files = [date_to_path(d) for d in datestrings]
files[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://edill-data/cleaned-logs/2016/10/2016-10-28.parquet'

In [132]:
sdf = spark.read.parquet(*files)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
grouped = (sdf.withColumn('total_bytes', F.col('bytes') * F.col('counts'))
          .withColumn('ymd', F.to_date(F.col('time')))
          .groupby('ymd', 'hour', 'data_source')
          .agg({'total_bytes': 'sum'})
          .withColumnRenamed('sum(total_bytes)', 'total_bytes')
      )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [134]:
table = 'default.downloads'
spark.sql('create table if not exists %s as select * from tmp_grouped ' % 
          (table))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [135]:
grouped.write.insertInto('default.downloads', overwrite=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [136]:
existing = spark.sql('select * from %s' % table)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [137]:
grouped.head()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(ymd=datetime.date(2019, 7, 16), hour=1, data_source=u'anaconda', sum(total_bytes)=1698914624779)

In [138]:
concat = (existing.union(grouped)
                  .drop_duplicates()
                  .write.insertInto(table, overwrite=True)
         )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [144]:
sdf = spark.sql('select * from %s' % table)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
sdf.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

73700

In [153]:
1039 * 24

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

24936

In [152]:
%%sql?

In [154]:
%%sql -n 100000 -o sdf
select * from default.downloads

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/home/ericdill/miniconda/envs/sparkmagic/lib/python3.6/site-packages/autovizwidget/widget/utils.py:50: FutureWarning:

A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.



Output()

In [156]:
%%local
len(sdf)

73700

In [158]:
%%local
sdf.to_csv('download_summary.csv')

In [ ]:
spark.stop()